# 1. Getting the data

### The main code for querying the Places API
After running this the object response will have the json we queried stored in pure text format.\
Some adjustments will be needed to get more than 20 items from the query, we can do this by using nextPageToken.

In [43]:
import requests, json, time
from secretfile import api_key

origin_location = "57.70867542111475,11.973303298399806"
radius_in_meters = 2000


# A list of restaurant types
restaurant_type = ["burger", "pizza", "thai", "italian", "chinese", "sushi", "grill", "taco", "vietnamese", "tapas", "kebab", "café", "salad", "indian", "poke"]

# Payload and headers, empty for this query
payload={}
headers = {}

# The keys function as attributes in this dict
result_dict = {"name": [], "rating": [], "user_ratings_total": [], "formatted_address": [], "place_id": []}

# The keys function as entities in this dict


def get_request(url):
    response = requests.request("GET", url, headers=headers, data=payload)
    response.raise_for_status()
    return response

for query_string in restaurant_type:
    # The base url which we base our searches on
    url_textsearch = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_string}&type=restaurant&location={origin_location}&radius={radius_in_meters}&key={api_key}"
    response_to_dict(get_request(url_textsearch))

populate_openinghours()

NameError: name 'openinghours_dict' is not defined

### Querying for multiple types of restaurants

Will add code here for getting more results and from all types of restaurants

### Formatting the data we just got

We want to write our results to a file temporarily for convenience, then populate a dataframe with the data.\
We will use the PlaceID as a foreign key and create two different dataframes, one for the places themselves\
and one for their opening hours. For the restaurants that don't have opening hours values\
for a certain day we will want to have the value be "closed".

In [5]:
def response_to_dict(response):
    with open('restaurants.json', 'w' , encoding='utf-8') as w:
        w.write(response.text)
    # Populate the dict with values from the results
    with open('restaurants.json', 'r', encoding='utf-8') as data_file:
        data = json.load(data_file)
        for restaurant in data['results']:
            if restaurant['place_id'] not in result_dict['place_id'] and 'Picadeli Salladsbar' != restaurant['name']:
                result_dict['place_id'].append(restaurant['place_id'])
                result_dict['name'].append(restaurant['name'])
                result_dict['rating'].append(restaurant['rating'])
                result_dict['user_ratings_total'].append(restaurant['user_ratings_total'])
                result_dict['formatted_address'].append(restaurant['formatted_address'])
            else:
                pass

There is still some data that we have not been able to catch; the opening hours\
We will need to iterate through the names in a Place Details query against the Places API

In [42]:
def populate_openinghours():
# We query for the opening hours of every restaurant
    for place_id in result_dict['place_id']:
        url_findplace = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=opening_hours&key={api_key}"
        response = get_request(url_findplace)
        findplace_data = json.loads(response.text)

        # Since the data we get out form the query

        if 'opening_hours' in findplace_data['result']:
            openinghours_dict[place_id] = findplace_data['result']['opening_hours']['weekday_text']
        else:
            openinghours_dict[place_id] = "None" 

We will need to clean the opening_hours later, for now we will keep them in that format.

Now that we have the data we need to put it in a dataframe


In [2]:
import pandas as pd

In [ ]:
# Create a neat dataframe and sort the rows by rating, user_ratings_total
df_places = pd.DataFrame.from_dict(result_dict)
df_places.set_index('place_id')
df_places['rating']
df_places['user_ratings_total'] = df_places['user_ratings_total'].astype(int)
df_places = df_places[df_places['name'] != 'Picadeli Salladsbar']
df_places.sort_values(['rating','user_ratings_total'], ascending=False)

Now let's define some things

In [41]:
def grouped_ratings(rating):
    if rating >= 4.5:
        return 'Excellent'
    elif 4.5 > rating >= 3.75:
        return 'Great'
    elif 3.75 > rating >= 3.0:
        return 'Good'
    elif  3.0 > rating > 2.0:
        return 'Decent'
    else:
        return 'Bad'

Here I'm just testing some things with the rating_groups\
used this to find only the excellent restaurants with over 200 ratings

In [40]:
df_places['rating_group'] = df_places['rating'].apply(grouped_ratings)
df_excellent = df_places.loc[df_places['rating_group'] == 'Excellent']
df_excellent = df_excellent.loc[df_excellent['user_ratings_total'] > 200]
df_excellent.sort_values(['rating', 'user_ratings_total'], ascending=False)

,name,rating,user_ratings_total,formatted_address,place_id,rating_group
33,Buono Pizza,4.8,406,"Teatergatan 15, 411 35 Göteborg",ChIJ273XHHTzT0YR62jP7yAKvao,Excellent
60,La Cucina Italiana,4.7,323,"Skånegatan 33, 412 52 Göteborg",ChIJLURKSHbzT0YR9DuLi1AyS-I,Excellent
151,Restaurang Vi Viet,4.6,782,"Erik Dahlbergsgatan 20, 411 26 Göteborg",ChIJf6F-Em7zT0YR3mRP1lUM3co,Excellent
22,Pizzeria Monaco,4.6,543,"Friggagatan 25A, 416 64 Göteborg",ChIJ1dIDL37zT0YRyHLxkWmwcPA,Excellent
251,Geisha Sushi & Wok,4.6,308,"Nordenskiöldsgatan 1, 413 09 Göteborg",ChIJWVfr1BTzT0YRK5RS7WdlaMM,Excellent
101,Yoshi Sushibar,4.6,217,"Alströmergatan 3, 411 01 Göteborg",ChIJ4U_YQX7zT0YRK9R6P4QFMmw,Excellent
18,The Barn,4.5,2398,"Kyrkogatan 11, 411 15 Göteborg",ChIJK1IOMyqw6UMRzqyaMNgiVds,Excellent
9,2112,4.5,1923,"Magasinsgatan 5, 411 18 Göteborg",ChIJPSytJGbzT0YRKUOpf_qtaeM,Excellent
153,Duy-Tan,4.5,833,"Kvilletorget 19, 417 04 Göteborg",ChIJ-8kWTab0T0YR1khMAOxIddI,Excellent
253,Takame,4.5,690,"Sankt Eriksgatan 8, 411 05 Göteborg",ChIJQfgtiDPzT0YRUVlOT8wKRIw,Excellent


In [196]:
df_opening_hours = pd.DataFrame.from_dict(openinghours_dict, orient='index')
df_opening_hours

,0,1,2,3,4,5,6
ChIJsVTFD3DzT0YRH5WMvE6q5kQ,Monday: 11:30 AM – 12:00 AM,Tuesday: 11:30 AM – 12:00 AM,Wednesday: 11:30 AM – 12:00 AM,Thursday: 11:30 AM – 12:00 AM,Friday: 11:30 AM – 1:00 AM,Saturday: 12:00 PM – 1:00 AM,Sunday: 12:00 – 11:00 PM
ChIJq2LaBCjzT0YRi-8wBQsPM3E,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 7:00 PM
ChIJt2e12WXzT0YR1mzQqaFk6bc,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 8:00 PM
ChIJT_lXYH_zT0YRW104WmLMNzk,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 8:00 PM
ChIJ22Wom6_zT0YRroSDRonvRPc,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 12:00 – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 8:00 PM
...,...,...,...,...,...,...,...
ChIJddc7EjHyT0YRXaWcvQX8kfQ,Monday: 11:00 AM – 10:00 PM,Tuesday: 11:00 AM – 10:00 PM,Wednesday: 11:00 AM – 10:00 PM,Thursday: 11:00 AM – 10:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 11:00 AM – 10:00 PM,Sunday: 11:00 AM – 10:00 PM
ChIJzVLSi1LzT0YRGZo98DU9gAI,Monday: 11:00 AM – 7:00 PM,Tuesday: 11:00 AM – 7:00 PM,Wednesday: 11:00 AM – 7:00 PM,Thursday: 11:00 AM – 7:00 PM,Friday: 11:00 AM – 7:00 PM,Saturday: 12:00 – 6:00 PM,Sunday: 12:00 – 6:00 PM
ChIJXdr3gGbzT0YRuiYvZaTJWuY,Monday: 11:00 AM – 12:00 AM,Tuesday: 11:00 AM – 12:00 AM,Wednesday: 11:00 AM – 12:00 AM,Thursday: 11:00 AM – 12:00 AM,Friday: 11:00 AM – 1:00 AM,Saturday: 11:00 AM – 1:00 AM,Sunday: 11:00 AM – 10:00 PM
ChIJrUILuPnzT0YRbO6hFi83BF8,Monday: 11:00 AM – 10:00 PM,Tuesday: 11:00 AM – 10:00 PM,Wednesday: 11:00 AM – 10:00 PM,Thursday: 11:00 AM – 10:00 PM,Friday: 11:00 AM – 11:00 PM,Saturday: 1:00 – 11:00 PM,Sunday: 1:00 – 10:00 PM


### Storing the data

The route I went for was storing this data in a .csv file\
To do this we simply use the built in Pandas dataframe to_csv method\
We will store everything in two separate .csv files